In [1]:
import os
import gc
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' #Disable tensorflow debug info

from numpy import mean
from numpy import std
from numpy import asarray
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as plt
from pandas import merge
import pandas as pd
import datetime
import numpy as np
import pywt
import seaborn as sns

from tensorflow.keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D

from utils.load_data_utils import load_to_df, create_rolling_window_data
from utils.features_utils import add_svm_feature
from tensorflow.keras.utils import to_categorical


# Copied from internet to get rid of error
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

KeyboardInterrupt: 

In [8]:
accel_data_prefix = 'converted_data/'
train_tags = [3,5,8,9,10]
test_tags = [7]

# Array of all the training data we load in from each tag
train_inputs = []
train_groundtruths = []
# Array of all the testing data we load in from each tag
test_inputs = []
test_groundtruths = []

# Wavelet constants
waveletname = "mexh"
scales = range(1,201)

## Train Data

In [ ]:

# Load in all the training
for tag in train_tags:
    # Build folder path
    accel_data_dir = accel_data_prefix + 'T' + str(tag).zfill(2) + '/'
    groundtruth_dir = 'behavior_analysis/individual_behaviors/'

    # Get all accelerometer sensor data files for this folder
    accel_filepaths = os.listdir(accel_data_dir)
    accel_filepaths = [accel_data_dir + file for file in accel_filepaths if file.startswith('sensor_data') and file.endswith('0725.csv')]
    accel_filepaths.sort() # Make sure they're in order for processing
    
    # Get groundtruth path
    groundtruth_path = groundtruth_dir + 'C' + str(tag).zfill(2) + '_0725.csv'

    # Load in both sensor data
    accel_input_df, groundtruth_df = load_to_df(accel_filepaths, groundtruth_path)

    print(f"Loaded in tag {tag}")
    # Create sliding window
    X, y = create_rolling_window_data(accel_input_df, groundtruth_df,window_size=20, stride=10)
    print(f"Created Sliding window for tag {tag} \n")

    # Create wavelets
    wvs = np.ndarray((len(X),200,200,3))

    for wi in range(len(X)):
        # Transpose into X, Y, and Z channels
        window = X[wi].T
        for si in range(3):
            signal = window[si]
            coeff, _ = pywt.cwt(signal, scales, waveletname, 1)
            wvs[wi, :, :, si] = coeff
        print("Wavelet")


    # Add to array
    train_inputs.append(wvs)
    train_groundtruths.append(y)

    # Manage memory
    del [accel_input_df, groundtruth_df]
    gc.collect()

## Testing Data

In [12]:
# Load in all the testing
for tag in test_tags:
    # Build folder path
    accel_data_dir = accel_data_prefix + 'T' + str(tag).zfill(2) + '/'
    groundtruth_dir = 'behavior_analysis/individual_behaviors/'

    # Get all accelerometer sensor data files for this folder
    accel_filepaths = os.listdir(accel_data_dir)
    accel_filepaths = [accel_data_dir + file for file in accel_filepaths if file.startswith('sensor_data') and file.endswith('0725.csv')]
    accel_filepaths.sort() # Make sure they're in order for processing
    
    # Get groundtruth path
    groundtruth_path = groundtruth_dir + 'C' + str(tag).zfill(2) + '_0725.csv'

    # Load in both sensor data
    accel_input_df, groundtruth_df = load_to_df(accel_filepaths, groundtruth_path)

    print(f"Loaded in tag {tag}")
    # Create sliding window
    X, y = create_rolling_window_data(accel_input_df, groundtruth_df,window_size=20, stride=10)
    print(f"Created Sliding window for tag {tag} \n")

    # Create wavelets
    wvs = np.ndarray((len(X),200,200,3))

    for wi in range(len(X)):
        # Transpose into X, Y, and Z channels
        window = X[wi].T
        for si in range(3):
            signal = window[si]
            coeff, _ = pywt.cwt(signal, scales, waveletname, 1)
            wvs[wi, :, :, si] = coeff


    # Add to array
    test_inputs.append(wvs)
    test_groundtruths.append(y)

    # Manage memory
    del [accel_input_df, groundtruth_df]
    gc.collect()

/Users/maxleblang/Desktop/cow-posture-analysis/utils/load_data_utils.py:31: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  standardized_input.fillna(method='ffill', inplace=True)


Loaded in tag 7
Base time is: 1
Created Sliding window for tag 7 



## Build the Model

In [28]:
def CNN(wv_x, wv_y, wv_channels, num_classes):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(5, 5), strides=(1, 1),
                    activation='relu',
                    input_shape=(wv_x, wv_y, wv_channels)))
    model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Conv2D(64, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Flatten())
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam', metrics=['accuracy'])

    return model

In [ ]:
def get_confusion_matrix(model, testX, testy):
    y_pred = model.predict(testX)

    # One hot encoding back to single int
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_test_classes = np.argmax(testy, axis=1)

    conf_matrix = confusion_matrix(y_test_classes, y_pred_classes)

    plt.figure(figsize=(6, 4))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['1', '2', '3', '4','5','6','7'], yticklabels=['1', '2', '3', '4','5','6','7'])
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    plt.savefig('raw_accel_conf_matrix.png')

In [ ]:
# Train and evaluate a model
def build_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 0, 15, 32
    n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
    n_batches = trainX.shape[0]

    model = CNN(n_timesteps, n_features, n_outputs)
    # fit network
    model.fit(trainX, trainy, epochs=epochs,
              batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(
        testX, testy, batch_size=batch_size, verbose=0)
    
    get_confusion_matrix(model, testX, testy)
    
    return accuracy